## 데이터 수집

* archives.go.kr
* http://theme.archives.go.kr/next/closed/page/ReadClosedCommon.do?yr=2019

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import time
import numpy as np

In [2]:
url = "http://theme.archives.go.kr/next/closed/page/ReadClosedCommon.do?yr=2019"

In [3]:
response = requests.get(url)
html = bs(response.text)

In [4]:
table = pd.read_html(url)
len(table)

207

In [5]:
# body > table > tbody > tr:nth-child(2) > td:nth-child(2) > table > tbody > tr:nth-child(4) > td > table
url = "http://theme.archives.go.kr/next/closed/page/orgListClosedCommon.do"
query = "bookmark=&pageParam=page&page=2&searchResultCount="
query = f"{query}&gubun=80&orgnz_id=108001&searchType=&keyword=&keywordGubun=&viewPage=ClosedList2017"
query

'bookmark=&pageParam=page&page=2&searchResultCount=&gubun=80&orgnz_id=108001&searchType=&keyword=&keywordGubun=&viewPage=ClosedList2017'

In [6]:
q_split = query.split("&")
params = {}
for q in q_split:
    q_key = q.split("=")[0]
    q_val = q.split("=")[1]
    params[q_key] = q_val
    
params

{'bookmark': '',
 'pageParam': 'page',
 'page': '2',
 'searchResultCount': '',
 'gubun': '80',
 'orgnz_id': '108001',
 'searchType': '',
 'keyword': '',
 'keywordGubun': '',
 'viewPage': 'ClosedList2017'}

In [7]:
def requests_page(url, params):
    """
    URL과 query string 을 만들어 페이지에 접근합니다.
    여기에서는 post를 통해 url을 호출합니다.
    """
    response = requests.post(url, params)
    html = bs(response.text)
    return html

In [8]:
def find_last_page_no(html):
    """마지막 페이지 번호를 찾아옵니다."""
    find_last_page = html.select("a")[-1]["href"]
    last_page_no = re.sub("[^0-9]", "", find_last_page)
    last_page_no = int(last_page_no)
    return last_page_no

In [9]:
def parse_table(html):
    """기록목록을 가져옵니다."""
    table = html.select("table")
    doc = table[1].select("table")[2]
    df_doc = pd.read_html(str(doc))[0]
    cols = df_doc.loc[0].tolist()
    df_doc.columns = cols
    df_doc = df_doc.drop(0)
    return df_doc

In [10]:
def get_table(url, params):
    """
    url, params 를 넘기면 기록물 목록을 반환
    """
    html = requests_page(url, params)
    table = parse_table(html)
    return table

In [11]:
html = requests_page(url, params)
last_page_no = find_last_page_no(html)
last_page_no

34988

In [12]:
params

{'bookmark': '',
 'pageParam': 'page',
 'page': '2',
 'searchResultCount': '',
 'gubun': '80',
 'orgnz_id': '108001',
 'searchType': '',
 'keyword': '',
 'keywordGubun': '',
 'viewPage': 'ClosedList2017'}

In [33]:
def g2b_page_scraping(url, params, start_page_no, end_page_no):
    page_list = []
    for page_no in range(start_page_no, end_page_no+1):
        """
        0~1초 휴지시간을 갖고 페이지마다 돌며 수집
        """
        random_time = np.random.uniform(0, 1)
        time.sleep(random_time)
        
        params["page"] = page_no
        df_page = get_table(url, params)
        page_list.append(df_page)
        df = pd.concat(page_list)
        file_name = f"archives_go_kr_{params['gubun']}_{params['orgnz_id']}_{start_page_no}_{end_page_no}.csv"
        df.to_csv(file_name, index=False)
        
        if page_no % 100 == 0 :
            print(page_no)
        
    return df

In [34]:
url, params

('http://theme.archives.go.kr/next/closed/page/orgListClosedCommon.do',
 {'bookmark': '',
  'pageParam': 'page',
  'page': 5,
  'searchResultCount': '',
  'gubun': '80',
  'orgnz_id': '108001',
  'searchType': '',
  'keyword': '',
  'keywordGubun': '',
  'viewPage': 'ClosedList2017'})

In [ ]:
start_page_no = 1
end_page_no = 1000
# end_page_no = last_page_no

df = g2b_page_scraping(url, params, start_page_no, end_page_no) 

100
200
300
400
500
600
700
800
900
